In [ ]:
import torch
import argparse
import numpy as np
from src.PointNet import PrimitivesEmbeddingDGCNGn
from src.mean_shift import MeanShift
from src.segment_utils import rotation_matrix_a_to_b
from torch.optim import Adam

def train_model(cfg, points, device):
    num_channels = 3  # 채널 수 설정 (정규화된 점군 데이터)
    model = PrimitivesEmbeddingDGCNGn(
        embedding=True,
        emb_size=128,
        primitives=True,
        num_primitives=10,
        mode=0,
        num_channels=num_channels,
    )
    model = torch.nn.DataParallel(model, device_ids=[0])
    model.to(device)
    model.train()

    # 학습에 필요한 설정
    optimizer = Adam(model.parameters(), lr=1e-3)
    criterion = torch.nn.CrossEntropyLoss()  # 또는 다른 손실 함수

    # 데이터 준비
    points = torch.from_numpy(points).float().to(device)

    # 훈련 반복
    for epoch in range(cfg.num_epochs):
        optimizer.zero_grad()
        # 모델의 forward pass
        output = model(points)
        
        # 손실 계산
        loss = criterion(output, labels)  # labels는 세그멘트된 클러스터의 라벨
        
        # 역전파 및 최적화
        loss.backward()
        optimizer.step()

        # 과적합을 유도하기 위한 반복
        if epoch % 10 == 0:
            print(f'Epoch [{epoch}/{cfg.num_epochs}], Loss: {loss.item():.4f}')

    return model

if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # 설정값 파싱
    parser = argparse.ArgumentParser(description="Train the Segmentation Model")
    parser.add_argument("--path_in", type=str, default="../assets/xyz/impellerdata.xyz")
    parser.add_argument("--num_epochs", type=int, default=100)
    cfg = parser.parse_args()

    # 점군 데이터 로드 및 정규화
    points = np.loadtxt(cfg.path_in).astype(np.float32)
    points = normalize_points(points)

    # 모델 훈련
    trained_model = train_model(cfg, points, device)
    print("모델 훈련 완료!")
